In [1]:
from pathlib import Path

import numpy as np
import pandas
from sklearn import metrics
from Bio import SeqIO
from matplotlib import pyplot
import seaborn


def get_hie(seq_index):
    hie = {}
    for name in seq_index:
        record = seq_index[name]
        sf = '.'.join(record.description.split()[1].split('.')[:3])
        hie.setdefault(sf, [])
        hie[sf].append(record)
    return hie


scop95_idx = SeqIO.index('data/scop/astral-scopedom-seqres-gd-sel-gs-bib-95-2.07.fa', 'fasta')
scop95_hie = get_hie(scop95_idx)
scop100_idx = SeqIO.index('data/scop/astral-scopedom-seqres-gd-all-2.07-stable.fa', 'fasta')
scop100_hie = get_hie(scop100_idx)
deltablast_d = np.load('data/delta_scop95.npy', allow_pickle=True).item()
test_data = get_hie(SeqIO.index('evaluation.fasta', 'fasta'))

In [ ]:
%matplotlib inline

from io import StringIO

from Bio import SearchIO
from tqdm.auto import tqdm


seaborn.set_context('poster')
fig, axes = pyplot.subplots(figsize=(50, 50), nrows=10, ncols=10)


auc_d = {'Method': [], 'ROC_n': [], 'AUC_n': []}

for ROC_n in tqdm((10, 50, 100, 250, 500)):
    for idx, sf_sccs in enumerate(test_data):
        query = test_data[sf_sccs][0]
        ax = axes[int(idx/10), idx%10]
        ax.set_title(sf_sccs)
        all_tp_count = len(scop95_hie[sf_sccs])
        # Proposed 1
        hits = np.load(f'data/evaluation/delta_u50_50_u50_50_s95_500_evalue_sum_local/{query.id}.npy', allow_pickle=True)
        agora_fp, agora_tp = [0], [0]
        n, i = 0, 0
        while n < ROC_n:
            if i < len(hits) and query.description.split()[1].split('.')[:3] == scop100_idx[hits[i][-3]].description.split()[1].split('.')[:3]:
                agora_tp.append(agora_tp[-1] + 1)
                agora_fp.append(agora_fp[-1])
            else:
                agora_fp.append(agora_fp[-1] + 1)
                agora_tp.append(agora_tp[-1])
                n += 1
            i += 1
        auc_d['Method'].append('2-Layers')
        auc_d['ROC_n'].append(ROC_n)
        auc_d['AUC_n'].append(metrics.auc(agora_fp, agora_tp) / ROC_n / all_tp_count)
        # Proposed 2
        if Path(f'data/evaluation/delta_u50_50_s95_500_evalue_sum_i_blast/{query.id}.npy').exists():
            hits = np.load(f'data/evaluation/delta_u50_50_s95_500_evalue_sum_i_blast/{query.id}.npy', allow_pickle=True)
            agora2_fp, agora2_tp = [0], [0]
            n, i = 0, 0
            while n < ROC_n:
                if i < len(hits) and query.description.split()[1].split('.')[:3] == scop100_idx[hits[i][4]].description.split()[1].split('.')[:3]:
                    agora2_tp.append(agora2_tp[-1] + 1)
                    agora2_fp.append(agora2_fp[-1])
                else:
                    agora2_fp.append(agora2_fp[-1] + 1)
                    agora2_tp.append(agora2_tp[-1])
                    n += 1
                i += 1
            auc_d['Method'].append('1-Layer')
            auc_d['ROC_n'].append(ROC_n)
            auc_d['AUC_n'].append(metrics.auc(agora2_fp, agora2_tp) / ROC_n / all_tp_count)
        # DELTA-BLAST
        deltablast_fp, deltablast_tp = [0], [0]
        results = SearchIO.read(StringIO(deltablast_d[query.id]), 'blast-xml')
        n = 0
        for hit in results.hits:
            if n == ROC_n: break
            if query.description.split()[1].split('.')[:3] == scop100_idx[hit.id].description.split()[1].split('.')[:3]:
                deltablast_tp.append(deltablast_tp[-1] + 1)
                deltablast_fp.append(deltablast_fp[-1])
            else:
                deltablast_fp.append(deltablast_fp[-1] + 1)
                deltablast_tp.append(deltablast_tp[-1])
                n += 1
        while n < ROC_n:
            deltablast_fp.append(deltablast_fp[-1] + 1)
            deltablast_tp.append(deltablast_tp[-1])
            n += 1
        auc_d['Method'].append('DELTA-BLAST')
        auc_d['ROC_n'].append(ROC_n)
        auc_d['AUC_n'].append(metrics.auc(deltablast_fp, deltablast_tp) / ROC_n / all_tp_count)
        ax.plot(agora_fp, agora_tp, linewidth=2)
        ax.plot(agora2_fp, agora2_tp, linestyle='dashdot')
        ax.plot(deltablast_fp, deltablast_tp, linestyle='dotted')
        fig.savefig('img/curve.pdf', bbox_inches='tight', pad_inches=0)

auc_df = pandas.DataFrame.from_dict(auc_d)
fig, ax = pyplot.subplots(figsize=(32, 9))
seaborn.boxplot(x='ROC_n', y='AUC_n', hue='Method', data=auc_df,
                ax=ax, showmeans=True, meanline=True, meanprops={'color': '#333333', 'linewidth': 2.5})
fig.savefig('img/roc_n.pdf', bbox_inches='tight', pad_inches=0)
fig, ax = pyplot.subplots(figsize=(32, 9))
seaborn.boxplot(x='Method', y='AUC_n', hue='ROC_n', data=auc_df,
                ax=ax, showmeans=True, meanline=True, meanprops={'color': '#333333', 'linewidth': 2.5})
fig.savefig('img/methods.pdf', bbox_inches='tight', pad_inches=0)

# # Diff
# desc = pandas.DataFrame.from_dict(auc_d, orient='index').transpose().describe()
# mean_df[sf_sccs] = {
#     'EvalueSumMin': desc['EvalueSumMin']['mean'],
#     'CloseCent': desc['CloseCent']['mean'],
#     'PageRank': desc['PageRank']['mean'],
#     'DELTA-BLAST': desc['DELTA-BLAST']['mean']
# }
# mean_diff['EvalueSumMin'].append(desc['EvalueSumMin']['mean'] - desc['DELTA-BLAST']['mean'])
# mean_diff['CloseCent'].append(desc['CloseCent']['mean'] - desc['DELTA-BLAST']['mean'])
# mean_diff['PageRank'].append(desc['PageRank']['mean'] - desc['DELTA-BLAST']['mean'])
# mean_df = pandas.DataFrame.from_dict(mean_df, orient='index')
# p3diff = mean_df['EvalueSumMin'] - mean_df['DELTA-BLAST']
# display(p3diff[p3diff <= -0.01])

# fig, ax = pyplot.subplots(figsize=(32, 9))
# seaborn.set_context('poster')
# seaborn.boxplot(data=pandas.DataFrame.from_dict(mean_diff, orient='index').transpose(),
#                 ax=ax, showmeans=True, meanline=True, orient='h', meanprops={'color': '#333333', 'linewidth': 2.5})
# ax.set(xlabel=f'AUC_{ROC_n} diff. with DELTA-BLAST')
# ax.axvline(color='gray', linestyle='dotted')
# #fig.savefig('boxplot.pdf', bbox_inches='tight', pad_inches=0)

In [6]:
auc_df[(auc_df['Method'] == '2-Layers') & (auc_df['ROC_n'] == 10)].describe()

,ROC_n,AUC_n
count,100.0,100.000000
mean,10.0,0.523283
std,0.0,0.349062
min,10.0,0.005525
25%,10.0,0.185952
50%,10.0,0.543092
75%,10.0,0.863674
max,10.0,1.000000


In [7]:
auc_df[(auc_df['Method'] == 'DELTA-BLAST') & (auc_df['ROC_n'] == 10)].describe()

,ROC_n,AUC_n
count,100.0,100.000000
mean,10.0,0.469387
std,0.0,0.341890
min,10.0,0.005435
25%,10.0,0.130303
50%,10.0,0.447899
75%,10.0,0.808409
max,10.0,1.000000


In [8]:
auc_df[(auc_df['Method'] == '2-Layers') & (auc_df['ROC_n'] == 500)].describe()

,ROC_n,AUC_n
count,100.0,100.000000
mean,500.0,0.631833
std,0.0,0.336858
min,500.0,0.014811
25%,500.0,0.364480
50%,500.0,0.734421
75%,500.0,0.950943
max,500.0,1.000000


In [9]:
auc_df[(auc_df['Method'] == 'DELTA-BLAST') & (auc_df['ROC_n'] == 500)].describe()

,ROC_n,AUC_n
count,100.0,100.000000
mean,500.0,0.546183
std,0.0,0.344129
min,500.0,0.038109
25%,500.0,0.229112
50%,500.0,0.574267
75%,500.0,0.893584
max,500.0,1.000000
